In [17]:
import pickle
import numpy as np
import networkx as nx
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [18]:
# Fonction pour charger un fichier .pkl
def load_pkl_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

# Exemple de chemin de fichier .pkl
file_path = "dataset_train.pkl"  # Remplacez par le chemin de votre fichier .pkl

data = load_pkl_file(file_path)


In [19]:
print(data.iloc[0]["flux_dégradé"])  # Affiche la première paire de matrices

                  Abbesses  Aimé Césaire  Alexandre Dumas  Alma - Marceau  \
Abbesses               0.0           0.0              0.0             0.0   
Aimé Césaire           0.0           0.0              0.0             0.0   
Alexandre Dumas        0.0           0.0              0.0             0.0   
Alma - Marceau         0.0           0.0              0.0             0.0   
Alésia                 0.0           0.0              0.0             0.0   
...                    ...           ...              ...             ...   
Wagram                 0.0           0.0              0.0             0.0   
École Militaire        0.0           0.0              0.0             0.0   
Église d'Auteuil       0.0           0.0              0.0             0.0   
Église de Pantin       0.0           0.0              0.0             0.0   
Étienne Marcel         0.0           0.0              0.0             0.0   

                  Alésia  Anatole France  Anvers  Argentine  Arts et Métier

In [20]:
adj_matrices = []
flux_matrices = []

for i, (G, flux) in enumerate(zip(data["plan_metro_degradé"], data["flux_dégradé"])):
    
    # Convertir le graphe en une matrice d'adjacence
    adj_matrix = nx.adjacency_matrix(G).todense()  # Convertir en format dense
    
    # Convertir la matrice d'adjacence en numpy
    adj_matrix_np = np.array(adj_matrix)
    
    # Convertir la matrice de flux en numpy
    flux_matrix_np = np.array(flux)
    
    # Convertir les matrices en tensor PyTorch
    adj_tensor = torch.tensor(adj_matrix_np, dtype=torch.float32)
    flux_tensor = torch.tensor(flux_matrix_np, dtype=torch.float32)
    
    # Ajouter les matrices dans les listes
    adj_matrices.append(adj_tensor)  # Liste de matrices d'adjacence
    flux_matrices.append(flux_tensor)  # Liste de matrices de flux
    
    # Optionnel : Afficher les matrices d'adjacence et de flux pour chaque plan
    print(f"Plan de métro {i + 1} :")
    print(f"Adjacency Matrix Tensor :\n", adj_tensor)
    print(adj_tensor.shape)
    print(f"Flux Matrix Tensor :\n", flux_tensor)
    print(flux_tensor.shape)


Plan de métro 1 :
Adjacency Matrix Tensor :
 tensor([[0., 1., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([261, 261])
Flux Matrix Tensor :
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([261, 261])
Plan de métro 2 :
Adjacency Matrix Tensor :
 tensor([[0., 1., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([261, 261])
Flux Matrix Tensor :
 tensor([[0., 0., 0.,  ..., 0., 0., 

In [21]:
# Définition de la couche GCN
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, adj):
        # Application de la normalisation et propagation
        out = torch.matmul(adj, x)  # Multiplication par la matrice de connectivité
        out = self.linear(out)  # Transformation linéaire
        return F.relu(out)

# Définition du modèle GCN complet
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(input_dim, hidden_dim)  # Première couche
        self.gcn2 = GCNLayer(hidden_dim, output_dim)  # Deuxième couche

    def forward(self, x, adj):
        h = self.gcn1(x, adj)  # Propagation dans la première couche
        h = self.gcn2(h, adj)  # Propagation dans la deuxième couche
        return h  # Sortie du modè

In [22]:
class GATLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GATLayer, self).__init__()
        self.W = nn.Linear(in_features, out_features, bias=False)
        self.a = nn.Linear(2 * out_features, 1, bias=False)

    def forward(self, x, adj):
        h = self.W(x)
        N = h.size(0)
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1)
        e = F.leaky_relu(self.a(a_input).view(N, N))
        attention = F.softmax(e.masked_fill(adj == 0, -1e9), dim=1)
        h_prime = torch.matmul(attention, h)
        return F.relu(h_prime)

class GAT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GAT, self).__init__()
        self.gat1 = GATLayer(input_dim, hidden_dim)
        self.gat2 = GATLayer(hidden_dim, output_dim)

    def forward(self, x, adj):
        h = self.gat1(x, adj)
        h = self.gat2(h, adj)
        return h


In [ ]:
# Hyperparamètres
input_dim = 221  # Exemple : nombre d'attributs par station
hidden_dim = 64  # Dimension cachée
output_dim = 221  # Flux de passagers entre chaque paire de stations
learning_rate = 0.01

# Création du modèle GCN
model_gcn = GCN(input_dim, hidden_dim, output_dim)
model_gat = GAT(input_dim, hidden_dim, output_dim)

# Fonction de perte et optimiseur
criterion = nn.MSELoss()  # Erreur quadratique moyenne
optimizer1 = optim.Adam(model_gcn.parameters(), lr=learning_rate)
optimizer2 = optim.Adam(model_gat.parameters(), lr=learning_rate)

# Entraînement
epochs = 200

# Exemple de données d'entrée : liste de matrices de caractéristiques et d'adjacence
features_matrix = [torch.rand((input_dim, input_dim)) for _ in range(10)]  # 10 instances, chaque avec 221 stations et 10 caractéristiques
adj_list = [torch.rand((input_dim, input_dim)) for _ in range(10)]  # 10 matrices de connectivité de taille 221x221

# Exemple de cibles : matrices de flux réelles à prédire
y = [torch.rand((input_dim, input_dim)) for _ in range(10)]  # 10 matrices de flux réels de taille 221x221


In [25]:
for epoch in range(epochs):
    model_gcn.train()  # Mode entraînement
    optimizer1.zero_grad()  # Mise à zéro des gradients
    
    # Iterer à travers les matrices de caractéristiques et d'adjacence
    for i in range(len(features_matrix)):
        features = features_matrix[i]
        adj = adj_list[i]
        
        features = torch.tensor(features, dtype=torch.float32)  # Assurez-vous que 'features' est un Tensor
        adj = torch.tensor(adj, dtype=torch.float32)  # Assurez-vous que 'adj' est un Tensor
        
        # Propagation avant
        output = model_gcn(features, adj)  # Prédiction des flux de passagers pour l'instance i
        
        # Calcul de la perte pour chaque matrice de flux
        loss = criterion(output, y[i])
        
        # Rétropropagation et optimisation
        loss.backward()
        optimizer1.step()
    
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


print(output)

/tmp/ipykernel_6277/3601124850.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(features, dtype=torch.float32)  # Assurez-vous que 'features' est un Tensor
/tmp/ipykernel_6277/3601124850.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adj = torch.tensor(adj, dtype=torch.float32)  # Assurez-vous que 'adj' est un Tensor


Epoch 0, Loss: 0.3309551775455475
Epoch 20, Loss: 0.3309551775455475
Epoch 40, Loss: 0.3309551775455475
Epoch 60, Loss: 0.3309551775455475
Epoch 80, Loss: 0.3309551775455475
Epoch 100, Loss: 0.3309551775455475
Epoch 120, Loss: 0.3309551775455475
Epoch 140, Loss: 0.3309551775455475
Epoch 160, Loss: 0.3309551775455475
Epoch 180, Loss: 0.3309551775455475
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward0>)
